In [2]:
import geopandas as gpd
import rasterio
import os
from rasterio.plot import show
from rasterio import features
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import io
from rasterio.transform import from_origin
from rasterio.plot import show
from scipy.ndimage import distance_transform_edt
import mapclassify as mc
import numpy as np
from rasterio.transform import from_origin
import rasterio
from joblib import Parallel, delayed
import math

## Overview: AHP-based suitability modelling

This section applies the **Analytic Hierarchy Process (AHP)** to identify suitable areas for placing in-stream wetlands based on expert knowledge and environmental criteria.

AHP is a multi-criteria decision-making method that:
- Structures the decision problem hierarchically (goal → criteria → alternatives)
- Uses pairwise comparisons to assign weights to each criterion
- Aggregates weighted raster layers to produce a final suitability map

The same input variables used in the RF model (TWI, slope, SOC, clay, flow accumulation) are used here, but combined through a rule-based approach

### Calculating weights for AHP

In this step, we compute the **criterion weights** using the Analytic Hierarchy Process (AHP), based on a **pairwise comparison matrix**.

**Steps:**
1. **Load the reciprocal matrix** from a CSV file or define it directly in the notebook. The matrix should express the relative importance of each criterion using Saaty's 1–9 scale.
2. **Compute the principal eigenvector** of the matrix, which gives the relative weights of each criterion.
3. **Normalise the eigenvector** to ensure the weights sum to 1.
4. **Calculate the Consistency Index (CI)** and **Consistency Ratio (CR)** to check whether the comparisons are consistent.  
   - A CR value below **0.10** indicates an acceptable level of consistency.
5. **Display the final weights**, which will be used to weight the raster layers during map calculation.

In [4]:
# load your reciprocals matrix as a dataframe
# you can make it as a csv or using a dict in here, just make sure the result looks like this example
# (but do not use THIS example because I just made up the numbers)
# it doesn't matter which "direction" you calculate relative importance in, just make sure you are consistent
reciprocals = pd.read_csv('criteria.csv', index_col=0) #Pairwise Comparison Matrix
reciprocals

,slope,flow_acc,twi,soc,clay
slope,1.00,9.0,2.00,0.50,5.00
flow_acc,0.11,1.0,0.20,0.11,0.33
twi,0.50,5.0,1.00,0.14,3.00
soc,2.00,9.0,7.00,1.00,7.00
clay,0.20,3.0,0.33,0.14,1.00


In [5]:
# get principal eigenvector and eigenvalue
eigvals, eigvecs = np.linalg.eig(reciprocals)
principal_eigvec = eigvecs[:,0]
principal_eigval = eigvals[0] # AKA lambda max

# normalize principal eigenvector to get priority vector
# all complex components are zero so we can use np.real to get the real part
priority_vec = np.real(principal_eigvec / principal_eigvec.sum())

#these will be our weights
weights = pd.DataFrame({column: round(priority, 2) for column, priority in zip(reciprocals.index, priority_vec)}, index=['Weight'])
display(weights)

# calculate consistency index
# this should be less than 0.10
ci = (np.real(principal_eigval) - len(reciprocals)) / (len(reciprocals) - 1)
display(ci)
n=len(reciprocals)
ri=1.41 #ramdom index for n classes
cr=round((ci/ri),2)
if cr<=0.1:
    print('CR='+str(cr)+', Level of consistance is aceptable')
else:
    print('CR='+str(cr)+', Level of consistance is NOT aceptable')

,slope,flow_acc,twi,soc,clay
Weight,0.26,0.03,0.13,0.51,0.06


0.05149613021365207

CR=0.04, Level of consistance is aceptable


### Working with raster variables as NumPy arrays and exploring basic statistics

we load each environmental variable (e.g. slope,tw,flow acc, soc, clay) as a **NumPy array** from its corresponding raster file.

In this step:
- The raster is read using `rasterio`, and the first band is converted to a NumPy array.
- We check for the presence of **NaN values**, which represent missing data.
- Basic descriptive statistics (help to understand the data distribution) are calculated:
  - **Minimum and maximum values**
  - **25th, 50th (median), and 75th percentiles**

You must update the raster filename according to the **data source** (local or global) and **resolution** (10 m or 50 m) you are using.

Refer to the table below to select the correct raster file:

| Variable     | Local 10 m                | Local 50 m                | Global 10 m               | Global 50 m               |
|--------------|---------------------------|---------------------------|---------------------------|---------------------------|
| **TWI**      | `twi_10m_a.tif`           | `twi_50m_a.tif`           | `twi_10m_GEE_a.tif`       | `twi_50m_GEE_a.tif`       |
| **Slope**    | `slope_10m_a.tif`         | `slope_50m_a.tif`         | `slope_10m_GEE_a.tif`     | `slope_50m_GEE_a.tif`     |
| **Clay**     | `clay_10m_a.tif`          | `clay_50m_a.tif`          | `clay_10m_GEE_a.tif`      | `clay_50m_GEE_a.tif`      |
| **SOC**      | `soc_10m_a.tif`           | `soc_50m_a.tif`           | `soc_10m_GEE_a.tif`       | `soc_50m_GEE_a.tif`       |
| **Flow Acc.**| `flow_acc_10m_a.tif`      | `flow_acc_50m_a.tif`      | `flow_acc_10m_GEE_a.tif`  | `flow_acc_50m_GEE_a.tif`  |

Repeat this step for all variables you plan to include in your AHP model.

###  Slope

In [ ]:
#raster to numpy array
data_directory = 'data'
filename='slope_50m_a.tif'
input_raster =os.path.join(data_directory, filename)
# open raster with rasterio
with rasterio.open(input_raster) as src:
    # read the data with NumPy matrix 
    slope_array = src.read(1)  # read the first band as NumPy matrix 
    src.close()
    #print("Number of bands:", src.count)
    print("NumPy array dimensions:", slope_array.shape)
    print("min value:", np.nanmin(slope_array))
    print("max value:", np.nanmax(slope_array))

# nan values 
has_nans = np.isnan(slope_array).any()
print(has_nans)  

# Flatten the array and remove NaN values for calculations
slope_array_flat = slope_array.flatten()
slope_array_flat_no_nan = slope_array_flat[~np.isnan(slope_array_flat)]

# Calculate percentiles
print("25th Percentile (Slope):", np.percentile(slope_array_flat_no_nan, 25))
print("50th Percentile (Median) (Slope):", np.percentile(slope_array_flat_no_nan, 50))
print("75th Percentile (Slope):", np.percentile(slope_array_flat_no_nan, 75))

#### Selection criteria and suitability rating for in-stream wetland placement – *Slope*

The table below defines suitability classes for locating in-stream wetlands, based on slope values. Lower slopes are preferred, as they favour water retention and reduce flow velocity, making them more appropriate for wetland functioning within stream systems.

| **Suitability Class**        | **Slope Range (degrees)** |
|-----------------------------|:--------------------------:|
| 1 – No suitability           | *x* > 6                    |
| 2 – Marginally suitable      | 3 < *x* ≤ 6                |
| 3 – Moderately suitable      | 2 < *x* ≤ 3                |
| 4 – Highly suitable          | *x* ≤ 2                    |

These thresholds will be used to **reclassify the slope raster** into suitability scores prior to AHP overlay.

**Note**: These threshold values are based on **local 50 m resolution data**. If you are using data from a different source or resolution (e.g. global 10 m), you should explore your raster statistics and adjust the ranges accordingly.

In [ ]:
#Selection criteria and suitability rating 
tem1 = np.where(slope_array <= 2, 4, 0)
tem2 = np.where((slope_array > 2) & (slope_array <= 3), 3, 0)
tem3 = np.where((slope_array > 3) & (slope_array <= 6), 2, 0)
tem4 = np.where(slope_array > 6, 1, 0)

slope_array=tem1+tem2+tem3+tem4

#Replace 0s with NaN
slope_array = np.where(slope_array == 0, np.nan, slope_array)

print("NumPy array dimensions:", slope_array.shape)
print("Classification array min value:", np.nanmin(slope_array))
print("Classification array max value:", np.nanmax(slope_array))

NumPy array dimensions: (5138, 7403)
Classification array min value: 1.0
Classification array max value: 4.0


In [ ]:
##### numpy array to raster in the same location of the pixel 
src = rasterio.open(input_raster)
transform = src.transform
slope_array = slope_array.astype(np.float32) 
output_raster_path = 'reclassified_slope_50m.tif' 

#new raster
with rasterio.open(output_raster_path, 'w', driver='GTiff',
                   height=slope_array.shape[0], width=slope_array.shape[1],
                   count=1, dtype='float32',  
                   crs="EPSG:3301", transform=transform) as dst:
    # NumPy array to raster 
    dst.write(slope_array, 1)
    dst.close()

### Flow accumulation

In [ ]:
#raster to numpy array
data_directory = 'data'
filename='flow_acc_50m_a.tif'
input_raster =os.path.join(data_directory, filename)
# open raster with rasterio
with rasterio.open(input_raster) as src:
    # read the data with NumPy matrix 
    flow_acc_array = src.read(1)  # read the first band as NumPy matrix 
    print("NumPy array dimensions:", flow_acc_array.shape)
    print("min value:", np.nanmin(flow_acc_array))
    print("max value:", np.nanmax(flow_acc_array))
    
    # Assume negative values are NoData and replace with NaN
    flow_acc_array = np.where(flow_acc_array < 0, np.nan, flow_acc_array)
    print("Negative values have been replaced with NaN.")
    # Print the min and max values
    print("Min value (excluding NaN):", np.nanmin(flow_acc_array))
    print("Max value (excluding NaN):", np.nanmax(flow_acc_array))

# Flatten the array and remove NaN values for calculations
flow_acc_array_flat = flow_acc_array.flatten()
flow_acc_array_flat_no_nan = flow_acc_array_flat[~np.isnan(flow_acc_array_flat)]

# Print percentiles
print("25th Percentile:",  np.percentile(flow_acc_array_flat_no_nan, 25))
print("50th Percentile (Median):", np.percentile(flow_acc_array_flat_no_nan, 50))
print("75th Percentile:", np.percentile(flow_acc_array_flat_no_nan, 75))

NumPy array dimensions: (5138, 7403)
min value: -3195.4011
max value: 55781.043
Negative values have been replaced with NaN.
Min value (excluding NaN): 0.0043320963
Max value (excluding NaN): 55781.043
25th Percentile: 0.00458943285048008
50th Percentile (Median): 0.008123337291181087
75th Percentile: 0.029373185709118843


In [ ]:
##NATURAL BREAK ANALYSIS
# Flatten the array as Jenks Natural Breaks works on a 1D array
flow_acc_array_1d = flow_acc_array.flatten()
flow_acc_array_no_nan = flow_acc_array_1d[~np.isnan(flow_acc_array_1d)]
# Calculate the natural breaks for the data
breaks = mc.NaturalBreaks(flow_acc_array_no_nan, k=4)
print(f"Natural breaks (Class limits): {breaks.bins}")

#### Selection criteria and suitability rating for in-stream wetland placement – *Flow Accumulation*

The table defines suitability classes based on **flow accumulation**, which indicates the amount of upstream area contributing surface flow to a given location. Higher flow accumulation areas are more likely to intercept significant runoff

| **Suitability Class**        | **Flow Accumulation (x)** |
|-----------------------------|:--------------------------:|
| 1 – No suitability           | *x* < 5                    |
| 2 – Marginally suitable      | 5 ≤ *x* < 100              |
| 3 – Moderately suitable      | 100 ≤ *x* < 1000           |
| 4 – Highly suitable          | *x* ≥ 1000                 |

These thresholds are used to reclassify the flow accumulation raster into discrete suitability scores for integration into the AHP model.

**Note**: These threshold values are based on **local 50 m resolution data**. If you are using data from a different source or resolution (e.g. global 10 m), you should explore your raster statistics and adjust the ranges accordingly.


In [11]:
#Selection criteria and suitability rating 
tem1 = np.where(flow_acc_array >= 1000, 4, 0)
tem2 = np.where((flow_acc_array >= 100) & (flow_acc_array < 1000), 3, 0)
tem3 = np.where((flow_acc_array >= 5) & (flow_acc_array < 100), 2, 0)
tem4 = np.where(flow_acc_array < 5, 1, 0)

flow_acc_array=tem1+tem2+tem3+tem4

# Replace 0s with NaN
flow_acc_array = np.where(flow_acc_array == 0, np.nan, flow_acc_array)

print("NumPy array dimensions:", flow_acc_array.shape)
print("Classification array min value:", np.nanmin(flow_acc_array))
print("Classification array max value:", np.nanmax(flow_acc_array))

NumPy array dimensions: (5138, 7403)
Classification array min value: 1.0
Classification array max value: 4.0


In [28]:
##### numpy array to raster  
src = rasterio.open(input_raster)
transform = src.transform
flow_acc_array = flow_acc_array.astype(np.float32)
output_raster_path = 'reclassified_flow_acc_50m.tif'  
#Create a new raster using rasterio with float32 data type
with rasterio.open(output_raster_path, 'w', driver='GTiff',
                   height=flow_acc_array.shape[0], width=flow_acc_array.shape[1],
                   count=1, dtype='float32',  # Ensure the dtype is set to float32
                   crs="EPSG:3301", transform=transform) as dst:
    # Write the NumPy array to the new raster
    dst.write(flow_acc_array, 1)
    dst.close()

### Clay content

In [ ]:
#raster to numpy array
data_directory = 'data'
filename= 'clay_50m_a.tif'
input_raster =os.path.join(data_directory, filename)
# open raster with rasterio
with rasterio.open(input_raster) as src:
    # read the data with NumPy matrix 
    clay_array = src.read(1)  # read the first band as NumPy matrix 
    print("NumPy array dimensions:", clay_array.shape)
    print("min value:", np.nanmin(clay_array))
    print("max value:", np.nanmax(clay_array))

NumPy array dimensions: (5138, 7403)
min value: 5.0
max value: 70.0


In [ ]:
####NATURAL BREAK ANALYSIS
# Flatten the array as Jenks Natural Breaks works on a 1D array
clay_array_1d = clay_array.flatten()
clay_array_no_nan = clay_array_1d[~np.isnan(clay_array_1d)]
# Calculate the natural breaks for the data
breaks = mc.NaturalBreaks(clay_array_no_nan, k=4)
print(f"Natural breaks (Class limits): {breaks.bins}")

#### Selection criteria and suitability rating for in-stream wetland placement – *Clay Content*

Clay content influences soil permeability and water retention. Higher clay percentages are generally more suitable for wetland placement, as they help retain moisture and support hydrological functioning.

| **Suitability Class**        | **Clay Content (% by weight)** |
|-----------------------------|:-------------------------------:|
| 1 – No suitability           | *x* < 11                        |
| 2 – Marginally suitable      | 11 ≤ *x* < 27                   |
| 3 – Moderately suitable      | 27 ≤ *x* < 38                   |
| 4 – Highly suitable          | *x* ≥ 38                        |

These thresholds are based on **local 50 m resolution data**. If you are using a different dataset (e.g. global 10 m), explore the raster's distribution and adjust the ranges accordingly.

In [13]:
#Selection criteria and suitability rating 
tem1 = np.where(clay_array >=38, 4, 0)
tem2 = np.where((clay_array >= 27) & (clay_array < 38), 3, 0)
tem3 = np.where((clay_array >= 11) & (clay_array < 27), 2, 0)
tem4 = np.where(clay_array < 11, 1, 0)

clay_array=tem1+tem2+tem3+tem4

# Replace 0s with NaN
clay_array = np.where(clay_array == 0, np.nan, clay_array)

print("NumPy array dimensions:", clay_array.shape)
print("Classification array min value:", np.nanmin(clay_array))
print("Classification array max value:", np.nanmax(clay_array))

NumPy array dimensions: (5138, 7403)
Classification array min value: 1.0
Classification array max value: 4.0


In [31]:
##### numpy array to raster  
from rasterio.transform import from_origin
from rasterio.plot import show

src = rasterio.open(input_raster)
transform = src.transform
clay_array = clay_array.astype(np.float32)
output_raster_path = 'reclassified_clay_50m.tif'

# new raster using rasterio
with rasterio.open(output_raster_path, 'w', driver='GTiff',
                   height=clay_array.shape[0], width=clay_array.shape[1], 
                   count=1, dtype='float32',
                   crs="EPSG:3301", transform=transform) as dst:
#write the NumPy matrix in the new raster
    dst.write(clay_array, 1)
    dst.close()


### Soil organic content

In [ ]:
data_directory = 'data'
filename= 'soc_50m_a.tif'
input_raster =os.path.join(data_directory, filename)

#open raster with rasterio
with rasterio.open(input_raster) as src:
    # read the data with NumPy matrix 
    soc_array = src.read(1)   
    # Convert from decigrams per kilogram (dg/kg) to grams per kilogram (g/kg)
    soc_array = soc_array # by dividing by 10 (since 1 dg = 0.1 g)
    print("NumPy array dimensions:", soc_array.shape)
    print("min value:", np.nanmin(soc_array))
    print("max value:", np.nanmax(soc_array))

NumPy array dimensions: (5138, 7403)
min value: 0.32780153
max value: 35.03709


In [ ]:
###NATURAL BREAK ANALYSIS
# Flatten the array as Jenks Natural Breaks works on a 1D array
soc_array_1d = soc_array.flatten()
soc_array_no_nan = soc_array_1d[~np.isnan(soc_array_1d)]
# Calculate the natural breaks for the data
breaks = mc.NaturalBreaks(soc_array_no_nan, k=4)
print(f"Natural Breaks (Class Limits): {breaks.bins}")

In [ ]:
###STADISTICAL ANALYSIS
# Flatten the array and remove NaN values for percentile calculations
soc_array_flat = soc_array.flatten()
soc_array_flat_no_nan = soc_array_flat[~np.isnan(soc_array_flat)]
# Calculate percentiles
p25 = np.percentile(soc_array_flat_no_nan, 25)
p50 = np.percentile(soc_array_flat_no_nan, 50)
p75 = np.percentile(soc_array_flat_no_nan, 75)
print("25th percentile (%):", p25)
print("50th percentile (Median) (%):", p50)
print("75th percentile (%):", p75)

#### Selection criteria and suitability rating for in-stream wetland placement – *Soil Organic Carbon (SOC)*

Soil organic carbon is a key indicator of soil quality and moisture-holding capacity.

| **Suitability Class**        | **SOC (% by weight)**     |
|-----------------------------|:--------------------------:|
| 1 – No suitability           | *x* ≤ 9                    |
| 2 – Marginally suitable      | 9 < *x* ≤ 15               |
| 3 – Moderately suitable      | 15 < *x* ≤ 27              |
| 4 – Highly suitable          | *x* > 27                   |

These thresholds are based on **local 50 m resolution data**. If using other data sources or resolutions, check the data range and adjust the values accordingly.

In [15]:
#Selection criteria and suitability rating 
tem1 = np.where(soc_array >27, 4, 0)
tem2 = np.where((soc_array > 15) & (soc_array <= 27), 3, 0)
tem3 = np.where((soc_array > 9) & (soc_array <= 15), 2, 0)
tem4 = np.where(soc_array <= 9, 1, 0)

soc_array=tem1+tem2+tem3+tem4

# Replace 0s with NaN
soc_array = np.where(soc_array == 0, np.nan, soc_array)

print("NumPy array dimensions:", soc_array.shape)
print("Classification array min value:", np.nanmin(soc_array))
print("Classification array max value:", np.nanmax(soc_array))

NumPy array dimensions: (5138, 7403)
Classification array min value: 1.0
Classification array max value: 4.0


In [34]:
##### numpy array to raster  
src = rasterio.open(input_raster)
transform = src.transform
soc_array = soc_array.astype(np.float32)

output_raster_path = 'reclassified_soc_50m.tif'
# new raster using rasterio
with rasterio.open(output_raster_path, 'w', driver='GTiff', height=soc_array.shape[0], width=soc_array.shape[1],
                   count=1, dtype='float32', crs="EPSG:3301", transform=transform) as dst:
#write the NumPy matrix in the new raster
    dst.write(soc_array, 1)
    dst.close()

### Topographic Wetness Index (TWI)

In [ ]:
data_directory = 'data'
filename= 'twi_50m_a.tif'
input_raster =os.path.join(data_directory, filename)
# open raster with rasterio
with rasterio.open(input_raster) as src:
    # read the data with NumPy matrix 
    twi_array = src.read(1)  # read the first band as NumPy matrix 
    print("Dimensiones de la matriz NumPy:", twi_array.shape)
    print("Valor minimo:", np.nanmin(twi_array))
    print("Valor maximo:", np.nanmax(twi_array))

    twi_array = np.where(twi_array < 0, np.nan, twi_array)
    print("Negative values have been replaced with NaN.")
    
    # Print the min and max values
    print("Min value (excluding NaN):", np.nanmin(twi_array))
    print("Max value (excluding NaN):", np.nanmax(twi_array))

# Flatten the array and remove NaN values for calculations
twi_array_flat = twi_array.flatten()
twi_array_flat_no_nan = twi_array_flat[~np.isnan(twi_array_flat)]

# Print percentiles
print("25th Percentile:",  np.percentile(twi_array_flat_no_nan, 25))
print("50th Percentile (Median):", np.percentile(twi_array_flat_no_nan, 50))
print("75th Percentile:", np.percentile(twi_array_flat_no_nan, 75))

Dimensiones de la matriz NumPy: (5138, 7403)
Valor minimo: -99999.0
Valor maximo: 16.41423
Negative values have been replaced with NaN.
Min value (excluding NaN): 2.3344202
Max value (excluding NaN): 16.41423
25th Percentile: 8.996185302734375
50th Percentile (Median): 9.763982772827148
75th Percentile: 10.427072525024414


#### Selection criteria and suitability rating for in-stream wetland placement – *Topographic Wetness Index (TWI)*

The Topographic Wetness Index (TWI) reflects the potential for water accumulation in the landscape. Higher TWI values typically indicate wetter areas.

| **Suitability Class**        | **TWI (x)**                |
|-----------------------------|:--------------------------:|
| 1 – No suitability           | *x* ≤ 6                    |
| 2 – Marginally suitable      | 6 < *x* ≤ 9                |
| 3 – Moderately suitable      | 9 < *x* ≤ 11               |
| 4 – Highly suitable          | *x* > 11                   |

These thresholds are based on **local 50 m resolution data**. For other data sources or resolutions, explore the distribution of TWI values and adjust the ranges accordingly.

In [4]:
#Selection criteria and suitability rating 
tem1 = np.where(twi_array >11, 4, 0)
tem2 = np.where((twi_array > 9) & (twi_array <= 11), 3, 0)
tem3 = np.where((twi_array > 6) & (twi_array <= 9), 2, 0)
tem4 = np.where(twi_array <= 6, 1, 0)

twi_array=tem1+tem2+tem3+tem4

# Replace 0s with NaN
twi_array = np.where(twi_array == 0, np.nan, twi_array)

print("NumPy array dimensions:", twi_array.shape)
print("Classification array min value:", np.nanmin(twi_array))
print("Classification array max value:", np.nanmax(twi_array))

NumPy array dimensions: (5138, 7403)
Classification array min value: 1.0
Classification array max value: 4.0


In [5]:
##### numpy array to raster  
from rasterio.transform import from_origin
from rasterio.plot import show

src = rasterio.open(input_raster)
transform = src.transform
twi_array = twi_array.astype(np.float32)

output_raster_path = 'reclassified_twi_50m.tif'
# new raster 
with rasterio.open(output_raster_path, 'w', driver='GTiff',
                   height=twi_array.shape[0], width=twi_array.shape[1],
                   count=1, dtype='float32',
                   crs="EPSG:3301", transform=transform) as dst:
#write the NumPy matrix in the new raster
    dst.write(twi_array, 1)
    dst.close()

### AHP Suitability Map Creation

In this step, we compute the final AHP suitability map by combining the weighted raster layers of all input criteria.

Process:
- Each reclassified raster array (e.g. slope, TWI, SOC, clay, flow accumulation) is multiplied by its corresponding **AHP weight**.
- The weighted layers are then summed to produce a continuous suitability map.
- Areas with missing data (NaN values) are preserved in the final output.
- The resulting map is saved as a GeoTIFF (`AHP_local_50m_final.tif`) using the spatial reference of one of the input rasters.

This example uses **local data at 50 m resolution**. If using a different dataset, be sure to adjust the raster filenames and verify that weights match the variables being used.

In [ ]:
print(weights) #From AHP 

        slope  flow_acc   twi   soc  clay
Weight   0.26      0.03  0.13  0.51  0.06


In [17]:
# List of tuples containing the data arrays and their corresponding column names in 'weights'
data_items = [
    (slope_array, 'slope'),
    (flow_acc_array, 'flow_acc'),
    (clay_array, 'clay'),
    (soc_array, 'soc'),
    (twi_array, 'twi'),
]

# Initialize the suitability map to zero
suitability_map = 0

# Loop through each item, multiply the data array by its corresponding weight, and add to the suitability map
for data_array, weight_key in data_items:
    suitability_map += data_array * weights.iloc[0][weight_key]

suitability_map[np.isnan(flow_acc_array)]=np.nan

# Now suitability_map holds the computed value
src = rasterio.open('reclassified_slope_50m.tif')
transform = src.transform
output_raster_path = 'AHP_local_50m_final.tif'
# new raster 
with rasterio.open(output_raster_path, 'w', driver='GTiff',
                   height=suitability_map.shape[0], width=suitability_map.shape[1],
                   count=1, dtype=suitability_map.dtype,
                   crs=src.crs, transform=transform) as dst:
#write the NumPy matrix in the new raster
    dst.write(suitability_map, 1)
    dst.close()
print("NumPy array dimensions:", suitability_map.shape)
print("Classification array min value:", np.nanmin(suitability_map))
print("Classification array max value:", np.nanmax(suitability_map))

NumPy array dimensions: (5138, 7403)
Classification array min value: 0.9899999952316285
Classification array max value: 3.959999980926514
